In [1]:
import chromadb
chroma_client = chromadb.Client()

In [3]:
collection = chroma_client.create_collection(name="iqs")

In [4]:
collection.add(
    documents=["This is a document", "This is another document"],
    metadatas=[{"source": "my_source"}, {"source": "my_source"}],
    ids=["id1", "id2"]
)

C:\Users\rosha\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [02:33<00:00, 542kiB/s]   


In [12]:
print(collection.query(
    query_texts=["When did India get Independence"],
    n_results=2
))

{'ids': [['id3', 'id2']], 'distances': [[0.254454106092453, 2.057119846343994]], 'metadatas': [[None, {'source': 'my_source'}]], 'embeddings': None, 'documents': [['India got independence on 1947', 'This is another document']], 'uris': None, 'data': None}


In [10]:
collection.add(documents = ['India got independence on 1947'], ids = ['id3'])

In [13]:
from langchain_community.document_loaders import TextLoader


In [15]:
raw_documents = TextLoader('test.txt').load()


In [16]:
print(raw_documents)

[Document(page_content='blah blah', metadata={'source': 'test.txt'})]


In [26]:
collection = client.get_collection(name="iqs")

In [24]:
import chromadb

client = chromadb.Client()


In [29]:
print(collection.to_dict())

AttributeError: 'Collection' object has no attribute 'to_dict'

In [30]:
from langchain.retrievers import ContextualCompressionRetriever, CohereRagRetriever
from langchain.retrievers.document_compressors import CohereRerank
from langchain_community.embeddings import CohereEmbeddings
from langchain_community.chat_models import ChatCohere
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
import os, dotenv

In [48]:
user_query = "when was cohere founded"
dotenv.load_dotenv()
API_KEY_OK = os.getenv('COHERE_API_KEY')

# Create cohere's chat model and embeddings objects
cohere_chat_model = ChatCohere(cohere_api_key=API_KEY_OK)
cohere_embeddings = CohereEmbeddings(cohere_api_key=API_KEY_OK)

In [49]:
raw_documents = TextLoader('D:\code\shlokathon\intellectquest\\documents\\Linked_List.txt').load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)

In [50]:
persist_directory = 'db'

In [51]:
vectordb = Chroma.from_documents(documents=documents, embedding=cohere_embeddings, persist_directory=persist_directory)

In [52]:
vectordb.persist()

In [53]:
vectordb = None

In [54]:
vectordb = Chroma(persist_directory=persist_directory, embedding_function=cohere_embeddings)


In [55]:
input_docs = vectordb.as_retriever().get_relevant_documents(user_query)



In [57]:
rag = CohereRagRetriever(llm=cohere_chat_model)
docs = rag.get_relevant_documents(
    user_query,
    source_documents=input_docs,
)
# Print the documents
for doc in docs[:-1]:
    print(doc.metadata)
    print("\n\n" + doc.page_content)
    print("\n\n" + "-" * 30 + "\n\n")
# Print the final generation 
answer = docs[-1].page_content
print(answer)
# Print the final citations 
citations = docs[-1].metadata['citations']
print(citations)

TypeError: 'NoneType' object is not iterable